In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import preprocessing
import numpy as np

In [2]:
house_train = pd.read_csv('../datasets/house_prices/train.csv')
house_test  = pd.read_csv('../datasets/house_prices/test.csv')

### Some data preprocessing

In [3]:
all_data = pd.concat([house_train.loc[:,'MSSubClass':'SaleCondition'],
                     house_test.loc[:,'MSSubClass':'SaleCondition']])



In [4]:
house_train_X = dummies[:house_train.shape[0]]
house_train_y = np.log1p(house_train.SalePrice)
house_test2 = dummies[house_train.shape[0]:]

house_train_X.shape

(1460, 149)

### Stacking approach

In [5]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

In [6]:
X_train, X_test, y_train, y_test = train_test_split(house_train_X, 
                                                    house_train_y,
                                                    test_size=0.2, random_state=42)



In [7]:
regr1 = linear_model.LinearRegression()
regr2 = Ridge(alpha=10)
meta_regressor = GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, max_depth=1, 
                                           random_state=0, loss='ls')

regr1.fit(X_train, y_train)
pred1 = regr1.predict(X_test)
print(pred1.shape)

regr2.fit(X_train, y_train)
pred2 = regr2.predict(X_test)
print(pred2.shape)

first_levels_pred = pd.DataFrame({'pred1': pred1, 'pred2': pred2})
X_flp_train, X_flp_test, y_flp_train, y_flp_test = train_test_split(first_levels_pred, 
                                                    y_test,
                                                    test_size=0.5, random_state=42)
print(X_flp_train.shape)
meta_regressor.fit(X_flp_train, y_flp_train)
meta_pred = meta_regressor.predict(X_flp_test)
meta_regressor.score(X_flp_test, y_flp_test)

(292,)
(292,)
(146, 2)


/usr/local/lib/python3.5/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


0.87864581935967501

In [8]:
def submit(df, title='Solution.csv'):
    df.to_csv(title, index=False)

submit_pred1 = regr1.predict(house_test2)
submit_pred2 = regr2.predict(house_test2)
submit_flp = pd.DataFrame({'pred1': submit_pred1, 'pred2': submit_pred2})

pred = np.expm1(meta_regressor.predict(submit_flp))
to_submit = pd.DataFrame({"Id":house_test.Id, "SalePrice":pred})
submit(to_submit, title="ensemble_method.csv")